In [1]:
from pyscf import gto
from qiskit import QuantumCircuit
from qc_practice.simulator import simulator
from qc_practice.measure.hamiltonian import hamiltonian, pyscf_luncher
from qc_practice.measure.measure import measure
from qc_practice.simulator.simulator import QASM
from qc_practice.ansatz import UCCSD
from qc_practice.profile import Profile

# Define the molecule
mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')

info = pyscf_luncher(mol)
h_pauli = hamiltonian(info)
ansatz = UCCSD()
simulator = QASM()
profile = Profile()
profile.num_elec = info['num_elec']
profile.num_orb = info['num_orb']

# Define the quantum circuit
qc = QuantumCircuit(profile.num_orb*2, profile.num_orb*2)
for i in range(profile.num_elec//2):
    qc.x(i)
    qc.x(i + profile.num_elec)

def calculation(coeff):
    qc = QuantumCircuit(profile.num_orb*2, profile.num_orb*2)
    for i in range(profile.num_elec//2):
        qc.x(i)
        qc.x(i + profile.num_elec)

    ansatz.mapping(profile, coeff)
    ansatz.ansatz(qc, profile, coeff)

    energy = measure(qc, h_pauli, simulator, parallel=True)
    print(energy, end='\r', flush=True)
    return energy

for i in range(2):
    coeff = ansatz.generate_coeff(profile)
    optimized = ansatz.call_optimizer(calculation, coeff, 'powell')

    print('\nCompleted')
    print(f'Energy: {optimized.fun}\n')

-1.8918539658846627
Completed
Energy: -1.8941507995888263

-1.8928703639577544
Completed
Energy: -1.8938942097303384



In [2]:
from pyscf import gto
from qiskit import QuantumCircuit
from qc_practice.measure.hamiltonian import hamiltonian
from qc_practice.measure.measure import measure
from qc_practice.simulator import QASM
from qc_practice.ansatz import UCCSD
from qc_practice.profile import Profile

class Test:
    def __init__(self):
        mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
        self.profile = Profile(mol)
        self.h_pauli = hamiltonian(self.profile)
        self.ansatz = UCCSD()
        self.simulator = QASM()
        self.profile_list = []

    def calculation(self, coeff):
        qc = QuantumCircuit(profile.num_orb*2, profile.num_orb*2)
        for i in range(profile.num_elec//2):
            qc.x(i)
            qc.x(i + profile.num_elec)

        self.ansatz.mapping(profile, coeff)
        self.ansatz.ansatz(qc, profile, coeff)

        energy = measure(qc, self.h_pauli, self.simulator, parallel=True)
        print(energy, end='\r', flush=True)
        self.profile.energy_elec = energy
        self.profile.coeff = coeff
        self.profile.circuit = qc
        return energy

    def run(self, coeff):
        for i in range(2):
            coeff2 = coeff.copy()
            optimized = self.ansatz.call_optimizer(self.calculation, coeff2, 'powell')
            self.profile.energy_elec = optimized.fun
            self.profile.coeff = optimized.x
            self.profile_list.append(self.profile)
            for j in range(i):
                overlap = self.simulator.swap_test(self.profile_list[0], self.profile_list[1])
                print(overlap)

            print('\nCompleted')
            print(f'Energy: {optimized.fun}\n')

mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
profile = Profile(mol)

test = Test()
coeff = UCCSD().generate_coeff(profile)

test.run(coeff)

-1.8911868638494949
Completed
Energy: -1.8931880573304367

01.8888131748175159

Completed
Energy: -1.892821477774264



In [3]:
import time
from multiprocessing import Pool
from qiskit import QuantumCircuit
from qiskit_aer import AerProvider

backend = AerProvider().get_backend('qasm_simulator')

def single_measure(num):
    time.sleep(1)
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    # backend = AerProvider().get_backend('qasm_simulator')
    result = backend.run(qc, shots=100000).result().get_counts()
    print(num)
    print(f'{result}\n')

def parallel_measure():
    with Pool(2) as p:
        p.map(single_measure, range(2))

    single_measure(2)

parallel_measure()

/tmp/ipykernel_4850/792659838.py:6: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = AerProvider().get_backend('qasm_simulator')


In [1]:
from pyscf import gto
from qc_practice import VQE, VQD
from qc_practice.ansatz import UCCSD
from qc_practice.simulator import QASM, StateVector

mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
vqe = VQE(mol)
vqe.ansatz = UCCSD()
vqe.simulator = StateVector()
vqe.run()



Starting VQE Calculation

Ansatz: UCCSD
Simulator: StateVector
Optimizer: powell

State 0:
Iteration: 80, Converged!!         
Total Energy: -1.136189454

Elapsed time: 0:00:07


Singlet_0

In [1]:
from pyscf import gto
from qc_practice import VQE, VQD
from qc_practice.ansatz import UCCSD, UpCCGSD
from qc_practice.simulator import QASM, StateVector

mol = gto.M(atom = 'H 0 0 0; H 0 0 0.7', basis = 'sto-3g')
vqe = VQD(mol)
vqe.ansatz = UpCCGSD()
vqe.simulator = StateVector()
vqe.nstates = 4
vqe.run()



Starting VQD Calculation

Ansatz: UCCSD
Simulator: StateVector
Optimizer: powell
nstates: 4

State 0:
Iteration: 80, Converged!!         
Total Energy: -1.136189454

State 1:
[[0.98904421]]Energy: -1.117350825
[[0.85916814]]Energy: -1.009949371
[[0.85916677]]Energy: -1.009946953
[[0.45128704]]Energy: -0.672647112
[[0.19363494]]Energy: -0.459579751
[[0.07726596]]Energy: -0.363347703
[[0.03000657]]Energy: -0.324266335
[[0.01153414]]Energy: -0.308990528
[[0.00441626]]Energy: -0.303104413
[[0.00168839]] Energy: -0.300848633
[[0.00064512]] Energy: -0.299985920
[[0.00024643]] Energy: -0.299656233
[[9.41298801e-05]]rgy: -0.299530303
[[3.59477636e-05]]rgy: -0.299482196
[[1.37294221e-05]]rgy: -0.299463826
[[5.24249386e-06]]rgy: -0.299456811
[[2.00076685e-06]]rgy: -0.299454132
[[7.63824363e-07]]rgy: -0.299453110
[[2.91401516e-07]]rgy: -0.299452720
[[1.10861528e-07]]rgy: -0.299452571
[[4.22967985e-08]]rgy: -0.299452514
[[1.60271923e-08]]rgy: -0.299452493
[[6.06337939e-09]]rgy: -0.299452485
[[2.2

In [1]:
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)

statevector = np.asarray(Statevector(qc)).reshape(-1, 1)
print(statevector)

[[0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]
 [0.70710678+0.j]]


In [37]:
a = np.array([[0, 1], [1, 0]])
matrix = np.kron(a, a)
probability = float(np.dot(statevector.T, np.dot(matrix, statevector)).real)
print(probability)

0.9999999999999998
